In [ ]:
# import libraries
from cil.optimisation.algorithms import LADMM, PDHG
from cil.optimisation.functions import KullbackLeibler, WeightedL2NormSquared, MixedL21Norm, BlockFunction, IndicatorBox, TotalVariation
from cil.optimisation.operators import BlockOperator, GradientOperator, IdentityOperator
from cil.framework import ImageGeometry, AcquisitionGeometry
from cil.plugins.astra.operators import ProjectionOperator
from cil.utilities.display import show2D
from cil.plugins.ccpi_regularisation.functions import FGP_TV 

import matplotlib.pyplot as plt
import os, sys
import numpy as np

import tomophantom
from tomophantom import TomoP2D

In [ ]:
# Load a tomophantom image 
model = 12 # select a model number from the library
N = 256 # set dimension of the phantom
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

phantom2D_np = TomoP2D.Model(model, N, path_library2D)    
ig = ImageGeometry(voxel_num_x=N, voxel_num_y=N, voxel_size_x = 0.5, voxel_size_y = 0.5)

phantom2D = ig.allocate()
phantom2D.fill(phantom2D_np)

In [ ]:
# Create Acquisition Geometry
detectors =  int(np.sqrt(2)*N)
angles = np.linspace(0, np.pi, 90, dtype=np.float32)

ag = AcquisitionGeometry.create_Parallel2D()\
                        .set_angles(angles,angle_unit="radian")\
                        .set_panel(detectors, pixel_size=0.5)

In [ ]:
device = "gpu"

In [ ]:
# Create Projection Operator
A = ProjectionOperator(ig, ag, device )

In [ ]:
# Create projection data and corrupt with noise
np.random.seed(10)

data = A.direct(phantom2D)
noisy_data = ag.allocate()

eta = ag.allocate(5)
scale = 2
n1 = np.random.poisson( scale*(data.as_array() + eta.as_array()))/scale
noisy_data.fill(n1)

In [ ]:
show2D([phantom2D, data, noisy_data], num_cols=3, cmap="inferno")

In [ ]:
# Regularisation parameter
alpha = 0.5

In [ ]:
# Setup and run ADMM  algorithm
Grad = GradientOperator(ig)
K = BlockOperator(Grad, A)
g = BlockFunction(alpha * MixedL21Norm(), KullbackLeibler(b=noisy_data,eta=eta))
f = IndicatorBox(lower=0.0)

normK = K.norm()
sigma = 1.
tau = sigma/normK**2

admm = LADMM(f=f, g=g, operator=K, sigma=sigma, tau=tau,
           max_iteration=1000, update_objective_interval=200)
admm.run(verbose=1)  

In [ ]:
# Setup and run PDHG implicit algorithm
sigma = 1.
tau = 1./(sigma*normK**2)
pdhg = PDHG(f=g, g=f, operator=K, sigma=sigma, tau=tau,
           max_iteration=1000, update_objective_interval=200)
pdhg.run(verbose=2)

In [ ]:
show2D([phantom2D, admm.solution, pdhg.solution, (admm.solution-pdhg.solution).abs()], 
       title=["Ground Truth", "ADMM ", "PDHG","AbsDiff"], cmap="inferno")

plt.figure(figsize=(10,5))            
plt.plot(np.linspace(0,ig.shape[1],ig.shape[1]), phantom2D.as_array()[int(ig.shape[0]/2),:], label = 'GTruth')
plt.plot(np.linspace(0,ig.shape[1],ig.shape[1]), admm.get_output().as_array()[int(ig.shape[0]/2),:], label = 'ADMM reconstruction')
plt.plot(np.linspace(0,ig.shape[1],ig.shape[1]), pdhg.get_output().as_array()[int(ig.shape[0]/2),:], label = 'PDHG reconstruction')

plt.legend()
plt.title('Middle Line Profiles')
plt.show()